# Kweather 데이터에 대한 관찰 코드

https://www.kaggle.com/izzettunc/introduction-to-time-series-clustering

## 1. Parameter Setting

In [1]:
import sys
sys.path.append("../")
sys.path.append("../..")

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(14, 4)

# 1.Data ingestion parameter
import pandas as pd
from datetime import timedelta
duration_day = 1 # 1 or 7
start_time = pd.to_datetime("2021-02-04 00:00:00")
end_time = start_time + timedelta(days = duration_day)
duration = {'start_time':start_time, 'end_time':end_time}

db_name = 'air_indoor_경로당'
feature_name = 'in_co2'

# 2.Data manipulation parameter
freq_min = 60
import datetime
timedelta_frequency_min = datetime.timedelta(minutes= freq_min)
# 3. Nan Processing Parameter
# 3-1. NanLImitProcessingInfo = {'type':'ratio', 'ConsecutiveNanLimit':0.05, 'totalNaNLimit':1} # seconds
# 3-2. NanLImitProcessingInfo = {'type':'time', 'ConsecutiveNanLimit':1*60*60, 'totalNaNLimit':5*60*60} #seconds
# 3-3.
NanInfoForCleanData = {'type':'num', 'ConsecutiveNanLimit':1, 'totalNaNLimit':1}

## 1. dataset Prepration

In [2]:
from datetime import timedelta
import math
timedelta_frequency_sec = timedelta(minutes=freq_min)

from clust.ingestion.influx import influx_Client_v2 as influx_Client, bucket_data
from setting import influx_setting_KETI as ins
db_client = influx_Client.influxClient(ins.CLUSTDataServer2)
ms_list = db_client.measurement_list(db_name)
from Clust.clust.ingestion.mongo import mongoClient
mongodb_client = mongoClient.mongoClient(ins.CLUSTMetaInfo2)

for ms_name in ms_list:
    data = db_client.get_data_by_time(start_time, end_time, db_name, ms_name)

    if feature_name in list(data.columns):

        from Clust.clust.preprocessing.custom.simple import preprocessing_basic_for_clust_oneData
        from Clust.clust.ingestion.mongo import mongoClient
        
        from Clust.clust.transformation.general import dataframe
        
        input_d = data[[feature_name]]
        result = preprocessing_basic_for_clust_oneData(input_d, mongodb_client, db_name, timedelta_frequency_min)
        
        clean_data1 = dataframe.get_oneDF_with_oneFeature_from_multipleDF(result['imputed_data'], feature_name, duration, timedelta_frequency_min)
        from Clust.clust.quality.NaN import cleanData
        CMS = cleanData.CleanData()
        clean_data2 = CMS.get_cleanData_by_removing_column(clean_data1, NanInfoForCleanData) 

        if feature_name in clean_data2.columns:
            flag = 1
            feature_data = list(clean_data2[feature_name].replace({math.nan:None}))
        else: 
            flag = 0
            feature_data = list(clean_data1[feature_name].replace({math.nan:None}))

    else:
        flag=-1
        feature_data = []
        
    print(ms_name, feature_data, flag )

/home/jwmoon/.local/lib/python3.8/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    import "influxdata/influxdb/schema" schema.measurements(bucket: "air_indoor_경로당") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


1 ---> 0
ICL1L2000234 [None, None, None, None, None, None, None, None, None, None, 124.4074074074074, 472.76666666666665, 472.48333333333335, 481.31666666666666, 476.8666666666667, 472.01666666666665, 478.28333333333336, 502.2, 323.75, 489.4035087719298, 483.5438596491228, 477.52542372881356, 473.93220338983053, 469.76271186440675] 0
1 ---> 0
ICL1L2000235 [None, None, None, None, None, None, None, None, None, 365.1304347826087, 494.73333333333335, 458.26666666666665, 447.1166666666667, 445.06666666666666, 438.8813559322034, 432.1666666666667, 437.15, 265.18333333333334, 435.81666666666666, 439.43333333333334, 442.68333333333334, 443.1320754716981, 438.54237288135596, 438.21666666666664] 0
1 ---> 1
ICL1L2000236 [444.45, 440.8813559322034, 438.1525423728813, 436.8305084745763, 261.6166666666667, 435.06666666666666, 434.49152542372883, 433.1551724137931, 432.1864406779661, 430.9, 431.4, 434.3448275862069, 432.78333333333336, 432.8103448275862, 433.6333333333333, 433.140350877193, 257.3389

1 ---> 0
ICL1L2000262 [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 452.96153846153845, 430.5, 423.7, 418.0, 413.23333333333335, 409.28333333333336, 405.8666666666667] 0
1 ---> 0
ICL1L2000263 [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 496.54545454545456, 427.65, 407.76666666666665, 396.46666666666664, 393.01666666666665, 392.43333333333334, 391.5833333333333, 388.9166666666667, 386.7] 0
1 ---> 0
ICL1L2000264 [580.0344827586207, 581.4166666666666, 582.3833333333333, 578.9, 575.0666666666667, 572.4833333333333, 570.6333333333333, 570.5, 568.9833333333333, 745.7833333333333, 1141.8166666666666, 1297.1, 1115.5689655172414, 1347.5, None, None, None, None, None, None, None, None, None, None] 0
1 ---> 1
ICL1L2000265 [504.01666666666665, 504.3666666666667, 504.03333333333336, 503.03333333333336, 497.73333333333335, 494.35, 490.01666666666665, 489.3666666666667, 489.23333333333335, 561.83333